In [1]:
# import pandas as pd
# import os

# ruta = 'data' 

# dataframes = []

# for archivo in os.listdir(ruta):
#     if archivo.endswith('.parquet'):
#         df = pd.read_parquet(os.path.join(ruta, archivo))
#         dataframes.append(df)

# df_concatenado = pd.concat(dataframes, ignore_index=True)
# df_concatenado.to_csv('resultado.csv', index=False)

C:\Users\br0445\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# pip3 install googletrans==3.1.0a0

In [3]:
import pandas as pd
from datasets import load_dataset
from googletrans import Translator
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

## **Dataset**

In [ ]:
dataset = load_dataset('detox_dataset.csv')

## **Traductor**

In [ ]:
trans = Translator()
print(trans.translate('Pues hazme los deberes, furcia barata!').text)

Well, do my homework for me, you cheap whore!


## **Detoxificador**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)  # Ajusta num_labels según tu tarea

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

trainer.train()

In [ ]:
inputs = tokenizer("¡Eres un idiota, no sabes nada y siempre metes la pata!", return_tensors='pt')
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=1)

---

---

In [15]:
csv = pd.read_csv('detox_dataset.csv')
df = pd.DataFrame(csv)

trans = Translator()
# print(trans.translate(df.iloc[400]['toxic_sentence'], dest='en').text)
print(trans.translate("ታምራት ነገራ ፅንፈኞችን ከላይ እስከታች ሰብስበህ ሁለት ሰንበት ፀበልና ስልጠና ስጥልን", dest='en').text)

Tamrat Negara gathered the extremists from top to bottom and gave us two Sabbath prayers and training.


## **BUCLE DE TRADUCCIÓN** (26 mins)

In [10]:
csv_trans = pd.read_csv('detox_dataset_trans.csv')
df_trans = pd.DataFrame(csv_trans)

In [12]:
for index in range(len(df)):
    traductor = Translator()
    toxic_translated = traductor.translate(df.iloc[index]['toxic_sentence'], dest='en').text
    neutral_translated = traductor.translate(df.iloc[index]['neutral_sentence'], dest='en').text
    new_row = [toxic_translated, neutral_translated]
    df_trans.loc[len(df_trans)] = new_row
    # break

df_trans.to_csv('detox_dataset_trans.csv', index=False)